# Classify `Salim` using a simple deep NLP
I will try to get text embeddings of tweet then using distance-based to give prob. if tweet is salim.

## dataset
Here is using Natch's dataset 25/10/2020. For info see <https://github.com/nimmanarati/awesome-data-science>

In [12]:
from pathlib import Path

dir_store = Path('datasets') / "natch"

In [13]:
import pandas as pd
df = pd.read_csv(dir_store / "WeLoveTheKing.csv")

## quick EDA

In [7]:
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,label,language,...,friends_count,listed_count,favourites_count,statuses_count,created_at_user,profile_image_url_https,default_profile,default_profile_image,Unnamed: 50,Unnamed: 51
0,1466,1.319119e+18,1.319119e+18,1603337449000,2020-10-22 10:30:49,700,NaN,ท่าทีของกลุ่มที่ชูป้ายความดีสำเร็จรูป #เรารักส...,0.0,th,...,1569,112,17430,103210,2007-10-21 23:55:18,https://pbs.twimg.com/profile_images/109536683...,FALSE,FALSE,NaN,NaN
1,3547,1.318902e+18,1.318902e+18,1603285654000,2020-10-21 20:07:34,700,NaN,ท่องคาถา​ #เรารักสถาบันพระมหากษัตริย์ แล้วชูภา...,0.0,th,...,1569,112,17430,103210,2007-10-21 23:55:18,https://pbs.twimg.com/profile_images/109536683...,FALSE,FALSE,NaN,NaN
2,7547,1.318812e+18,1.318812e+18,1603264144000,2020-10-21 14:09:04,700,NaN,หากชาว​แนวทาง​ #เรารักสถาบันพระมหากษัตริย์ สละ...,0.0,th,...,1569,112,17430,103210,2007-10-21 23:55:18,https://pbs.twimg.com/profile_images/109536683...,FALSE,FALSE,NaN,NaN
3,7595,1.318810e+18,1.318810e+18,1603263893000,2020-10-21 14:04:53,700,NaN,ข้อเสนอ​ปฏิรูป​สถาบัน​ฯ ที่จะชักชวนชาวแนวทาง​ ...,0.0,th,...,1569,112,17430,103210,2007-10-21 23:55:18,https://pbs.twimg.com/profile_images/109536683...,FALSE,FALSE,NaN,NaN
4,7798,1.318807e+18,1.318807e+18,1603262992000,2020-10-21 13:49:52,700,NaN,แนวทาง​ #เรารักสถาบันพระมหากษัตริย์ ควรยินดีกั...,0.0,th,...,1569,112,17430,103210,2007-10-21 23:55:18,https://pbs.twimg.com/profile_images/109536683...,FALSE,FALSE,NaN,NaN


In [6]:
all(pd.isna(df.place))

True

place columns all na 🤷‍♂️

In [11]:
pd.isna(df.quote_url).value_counts()

True     3922
False    2176
Name: quote_url, dtype: int64

Many of them are original. not just a retweet. 😍

In [12]:
df.label.value_counts()

1.0    70
0.0    18
Name: label, dtype: int64

In [15]:
df_selected = (df
#     .pipe(lambda d: d[pd.isna(d.quote_url)])
    .pipe(lambda d: d[["tweet", "label"]])
    .dropna(subset=["label"])
)

In [16]:
df_selected.shape

(88, 2)

In [17]:
df_selected.label.value_counts()

1.0    70
0.0    18
Name: label, dtype: int64

In [33]:
df_selected.to_csv(dir_store / 'selected.csv', index=False, header=False)

export to scan thru.

## Get embedding

In [8]:
from pythainlp.ulmfit import document_vector
from fastai import *
from fastai.text import *
from fastai.text.data import TextDataLoaders

In [9]:
# Load Data Bunch
model_path = 'models/pretrained_wiki'
# data = load_data(MODEL_PATH, 'thwiki_itos.pkl')
dls_lm = TextDataLoaders.from_folder(model_path, is_lm=True, valid_pct=0.1)

KeyboardInterrupt: 

In [5]:
# Initialize language_model_learner
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1,
     qrnn=False, tie_weights=True, out_bias=True, output_p=0.25,
     hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

In [ ]:
learn = language_model_learner(data, AWD_LSTM, config=config,
                                   pretrained=False, **trn_args)

In [ ]:
document_vector('วันนี้วันดีปีใหม่', learn, data)

Ok, umm, fastai last version use another api (`load_data` -> `TextDataLoaders`) it seems not work let try เปย์ version

## AutoML


In [19]:
df_selected[df_selected.label==1].tweet.to_csv('positive.csv', index=False)

In [20]:
df_selected[df_selected.label==0].tweet.to_csv('negative.csv', index=False)

In [23]:
salim_files = []
for i, r in enumerate(df_selected[df_selected.label==1].tweet):
    fname = f'datasets/primer/salim/{i}.txt'
    with open(fname, 'w') as f:
        f.write(r)
    salim_files.append(fname)

In [24]:
non_salim_files = []
for i, r in enumerate(df_selected[df_selected.label==0].tweet):
    fname = f'datasets/primer/non_salim/{i}.txt'
    with open(fname, 'w') as f:
        f.write(r)
    non_salim_files.append(fname)

In [26]:
salim_files[0].replace('datasets', 'gs://salim-datasets')

'gs://salim-datasets/primer/salim/0.txt'

In [28]:
salim_files = [(f.replace('datasets', 'gs://salim-datasets'), 'salim') for f in salim_files]

In [29]:
non_salim_files = [(f.replace('datasets', 'gs://salim-datasets'), 'non_salim') for f in non_salim_files]

In [34]:
pd.DataFrame(salim_files+non_salim_files).to_csv('datasets/primer/metadata.csv', index=False, header=False)